In [47]:
import sys

sys.path.append("../")
import json
from time import sleep

import pandas as pd
import requests

from vook_db_v7.config import MAX_PAGE, REQ_URL, WANT_ITEMS_RAKUTEN, req_params

In [ ]:
band_list = [
    "ローリングストーンズ",
    "ジミヘン",
    "レッド ツェッペリン",
    "デヴィッドボウイ",
    "METALLICA",
    "Guns N’ Roses",
    "KISS",
    "IRON MAIDEN",
    # https://jamtrading.jp/blogs/jam/1024509/
    # 検索確認ようhttps://jp.mercari.com/search?keyword=t%E3%82%B7%E3%83%A3%E3%83%84%20%E3%82%A2%E3%82%A4%E3%82%A2%E3%83%B3%E3%83%A1%E3%82%A4%E3%83%87%E3%83%B3&order=desc&price_min=10000&sort=price&status=sold_out%7Ctrading
]

In [ ]:
cnt = 1
item = "Tシャツ"
band_name = "ローリングストーンズ"
keyword = " ".join([item, category])
print(f"keyword:{keyword}")

MAX_PAGE = 100

req_params["page"] = cnt
req_params["keyword"] = keyword
req_params["minPrice"] = 10000

df = pd.DataFrame(columns=WANT_ITEMS_RAKUTEN)

while True:
    req_params["page"] = cnt
    res = requests.get(REQ_URL, req_params)
    res_code = res.status_code
    res = json.loads(res.text)
    if res_code != 200:
        print(
            f"""
        ErrorCode -> {res_code}\n
        Error -> {res['error']}\n
        Page -> {cnt}"""
        )
    else:
        if res["hits"] == 0:
            print("返ってきた商品数の数が0なので、ループ終了")
            break
        tmp_df = pd.DataFrame(res["Items"])[WANT_ITEMS_RAKUTEN]
        df = pd.concat([df, tmp_df], ignore_index=True)
    if cnt == MAX_PAGE:
        print("MAX PAGEに到達したので、ループ終了")
        break
    # logger.info(f"{cnt} end!")
    cnt += 1
    # リクエスト制限回避
    sleep(1)
    print("Finished!!")

df_main = df.rename(
    columns={"itemName": "name", "itemPrice": "price", "itemUrl": "url"}
)
df_main["band_name"] = band_name